In [1]:
!pip install lightgbm

In [3]:
!git clone https://github.com/Keshfer/Chocolate-Rating-ML

Cloning into 'Chocolate-Rating-ML'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 77 (delta 40), reused 50 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 3.43 MiB | 19.28 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [4]:
cd Chocolate-Rating-ML

/content/Chocolate-Rating-ML


In [5]:
# Import approriate functions and packages
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import the dataframe
df = pd.read_csv('cacao_engineered.csv')

# Create X and y training and testing data
X = df.drop(['Rating'], axis=1)
y = df['Rating']

# Clean up columns of X so it works with the model
X.columns = [col.replace(" ", "_").replace(":", "_") for col in X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Index(['Unnamed__0', 'REF', 'Review_Date', 'Cocoa_Percent',
       'Company_Location_AF', 'Company_Location_AS', 'Company_Location_CA',
       'Company_Location_CEU', 'Company_Location_CR', 'Company_Location_EEU',
       'Company_Location_NA', 'Company_Location_OC', 'Company_Location_SA',
       'Company_Location_WEU', 'Broad_Bean_Origin_AF', 'Broad_Bean_Origin_AS',
       'Broad_Bean_Origin_CA', 'Broad_Bean_Origin_CR', 'Broad_Bean_Origin_NA',
       'Broad_Bean_Origin_OC', 'Broad_Bean_Origin_SA',
       'Broad_Bean_Origin_Unknown'],
      dtype='object')


In [6]:
# Start creating lgb regression model
# Create the lgb Dataset objects to store the training and testing data
train_data = lgb.Dataset(X_train, label = y_train)
test_data = lgb.Dataset(X_test, label = y_test, reference=train_data)

# Set parameters for the lgb regression model
# Objective: Set to regression so that the model tries to predict a regressive value (in this case, quality)
# Metric: The loss metric to be used for the model. Set to mse for now.
# Learning rate: How much to update the trees when refining
# Feature fraction: The percentage of features to take into account when making model, used to prevent overfitting
params = {
    'objective': 'regression',
    'metric': 'mse',
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

# Train the actual model, passes in the parameters, training data, the number of rounds to train
num_round = 100  # Number of boosting rounds
model = lgb.train(
    params,
    train_data,
    num_boost_round=num_round,
    valid_sets=[train_data, test_data],
    valid_names=['train', 'eval'],
)

# Predict on the test set, using the best iteration of the model for training
y_pred = model.predict(X_test, num_iteration = model.best_iteration)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001750 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 582
[LightGBM] [Info] Number of data points in the train set: 1436, number of used features: 21
[LightGBM] [Info] Start training from score 3.189763
MSE: 0.19547680507452356
